In [1]:
import torch
import numpy as np
from torchvision.datasets import mnist
from torch import nn,optim
from torch.autograd import variable
from torch.utils.data import DataLoader

In [2]:
def tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x-0.5)/0.5
    x = x.reshape((-1, 1))
    x = torch.from_numpy(x)
    return x

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
train_set = mnist.MNIST('./dataSet', train=True, transform=tf, download = True)
test_set = mnist.MNIST('./dataSet', train=False, transform=tf, download = True)

In [22]:
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=True)

In [23]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(784, 400), 
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(400, 200), 
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(200, 100), 
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Linear(100, 10), 
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

In [25]:
# net = net.to(device)
net = net().cuda()
net

net(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=True)
    (1): ReLU()
  )
  (layer3): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [30]:
losses = []
acces = []
eval_losses = []
eval_acces = []
for e in range(30):
    train_loss = 0
    train_acc = 0
    net.train()
    for im, label in train_data:
        im = im.view(im.size(0), -1)
        im = im.to(device)
        label = label.to(device)
        # 前向传播
        out = net(im)
        loss = criterion(out, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 记录误差
        train_loss += loss.item()
        # 计算分类的准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
 
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    # 在测试集上检验效果
    eval_loss = 0
    eval_acc = 0
    net.eval()
    for im, label in test_data:
        im = im.view(im.size(0), -1)
        im = im.to(device)
        label = label.to(device)
        out = net(im)
        loss = criterion(out, label)
        # 记录误差
        eval_loss += loss.item()
        # 记录准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch:{}, Train Loss:{:.6f}, Train Acc:{:.6f}, Eval Loss:{:.6f}, Eval Acc:{:.6f}'
          .format(e, train_loss/len(train_data), train_acc / len(train_data),
                 eval_loss / len(test_data), eval_acc / len(test_data)))

epoch:0, Train Loss:0.012784, Train Acc:0.997518, Eval Loss:0.100218, Eval Acc:0.970629
epoch:1, Train Loss:0.012330, Train Acc:0.997601, Eval Loss:0.071630, Eval Acc:0.979035
epoch:2, Train Loss:0.011084, Train Acc:0.998118, Eval Loss:0.075868, Eval Acc:0.979529
epoch:3, Train Loss:0.010515, Train Acc:0.998268, Eval Loss:0.076647, Eval Acc:0.978244
epoch:4, Train Loss:0.009866, Train Acc:0.998368, Eval Loss:0.074194, Eval Acc:0.979134
epoch:5, Train Loss:0.008980, Train Acc:0.998867, Eval Loss:0.081542, Eval Acc:0.978244
epoch:6, Train Loss:0.008740, Train Acc:0.998651, Eval Loss:0.100614, Eval Acc:0.971717
epoch:7, Train Loss:0.008310, Train Acc:0.998834, Eval Loss:0.078363, Eval Acc:0.979134
epoch:8, Train Loss:0.007499, Train Acc:0.999134, Eval Loss:0.081921, Eval Acc:0.978046
epoch:9, Train Loss:0.006935, Train Acc:0.999150, Eval Loss:0.075185, Eval Acc:0.978145
epoch:10, Train Loss:0.006570, Train Acc:0.999434, Eval Loss:0.076062, Eval Acc:0.979134
epoch:11, Train Loss:0.006100, 